# Import Libraries & Upload Files

In [1450]:
# !pip install dash
# !pip install dash_daq
# !pip install dash_bootstrap_components
# !pip install pyproj
# !pip install dash_bootstrap_templates


In [1451]:
import dash_daq as daq
import dash_bootstrap_components as dbc
import folium
import json
import math
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from folium.plugins import BeautifyIcon
from dash import dcc, html, Input, Output, State
# from google.colab import files
from folium.map import Popup
from branca.element import Template, MacroElement
from dash_bootstrap_templates import ThemeChangerAIO


In [1452]:
# input_uploaded = files.upload()
# input_file = list(input_uploaded.keys())[0]  # Dynamically get the file name
# input_data = json.loads(input_uploaded[input_file].decode("utf-8"))
with open('input 200310.json', encoding='utf-8') as f:
    input_data = json.load(f)


In [1453]:
# output_uploaded = files.upload()
# output_file = list(output_uploaded.keys())[0]  # Dynamically get the file name
# output_data = json.loads(output_uploaded[output_file].decode("utf-8"))
with open('output_input 200310.json', encoding='utf-8') as f:
    output_data = json.load(f)

# Base Plugin

## Base Model

### Abstract Model Class

### Common Model Class

In [1454]:
# Function to add offset to lat/lng if there are overlapping locations
def add_offset(lat, lng, index, base_offset=0.001):
    """
    Modify lat, lng to prevent duplication customer/ hub by adding index * base_offset.
    If there are 3 duplicated points, index = 0, 1, 2.
    Output:
        New lat & lng of the current point

    Example:
        >>> add_offset(10.0, 20.0, 2)
        (10.002, 20.002)
    """
    offset = base_offset * index
    return lat + offset, lng + offset

In [1455]:
def list_of_dicts_to_plain_dict(list_dict, keyword, valuename):
    """
    Transform list of dict into plain dict via keyword and valuename
    Input: [{
            "factoryAddress": "165",
            "pdAdress": "PD-165",
            "match": 1
            },
            {
            "factoryAddress": "168",
            "pdAdress": "PD-168",
            "match": 1
            }]
    Output: {"165": "PD-165", "168": "PD-168"]}
    """

    return {
        element[keyword]: element[valuename] for element in list_dict
    }

In [1456]:
def list_of_dicts_to_dict_of_dicts(list_dict, keyword):
    """
    Transform list of dict into dict of dict via value of a key.
    Input: [{ "locationCode":"ABC",
                "lat": 10.2,
                "lng": 105.1}]
    Output: { "ABC": {"lat": 10.2,
                    "lng": 105.1}}
    """

    return {
        element[keyword]: {
            key: value for key, value in element.items() if key != keyword
        } for element in list_dict
    }

In [1457]:
def list_of_dicts_to_dict_of_list(list_dict, keyword, valuename):
    """
    Transform list of dict into dict of list via via keyword and valuename
    Input: [{
            "depotAddress": "D165",
            "pdAdress": "PD-165",
            "match": 1
            },
            {
            "depotAddress": "D165.2",
            "pdAdress": "PD-165",
            "match": 1
            }]
    Output: {"PD-165": ["D165", "D165.2"]}
    """

    return {
        component[keyword]: [
            element[valuename] for element in list_dict if component[keyword] == element[keyword]
        ] for component in list_dict
    }

In [1458]:
def reverse_plain_dict_to_dict_of_list(input_dict):
    reverse_dict = {}
    for key, value in input_dict.items():
        reverse_dict.setdefault(value, []).append(key)
    return reverse_dict

In [1459]:
def calculate_IBtranspCost(price_data, depot_code):
    total_price = 0
    for transp_cost in price_data["IBtranspCost"]:
        if transp_cost["destCode"] == depot_code:
            total_price += transp_cost["price"]
    return total_price


In [1460]:
def calculate_OBtranspCost(price_data, depot_code):
    total_price = 0
    for transp_cost in price_data["OBtranspCost"]:
        if transp_cost["srcCode"] == depot_code:
            total_price += transp_cost["price"]
    return total_price


In [1461]:
# Hàm để tìm thông tin về sản phẩm và dung lượng từ location_from đến location_to trong dữ liệu depots
def find_product_info(location_from, location_to, depots_data):
    product_info = []

    # Lặp qua mỗi depot trong dữ liệu depots
    for depot in depots_data:
        depot_code = depot["depotCode"]

        # Nếu depot_code trùng với location_from hoặc location_to
        if depot_code == location_from or depot_code == location_to:
            # Lặp qua mỗi khách hàng của depot
            for customer in depot["customers"]:
                customer_code = customer["customerCode"]

                # Nếu customer_code trùng với location_from hoặc location_to
                if customer_code == location_from or customer_code == location_to:
                    # Lặp qua mỗi sản phẩm của khách hàng
                    for product in customer["products"]:
                        product_info.append({
                            "productCode": product["productCode"],
                            "capacity": product["capacity"]
                        })

    return product_info

In [1462]:
pd_in_factory = list_of_dicts_to_plain_dict(input_data["matrixConfig"]["mapPhysicDepotWithFactory"],
                                                    "typeOfFactoryByAdress",
                                                    "typeOfPDByAdress")

In [1463]:
pd_in_factory

{'153': 'PD-153',
 '165': 'PD-165',
 '146': 'PD-146',
 '061': 'PD-061',
 'CC': 'PD-CC',
 '163': 'PD-163',
 '166': 'PD-166',
 '145': 'PD-145',
 '160': 'PD-160',
 '150': 'PD-150',
 '162': 'PD-162'}

## Base View

### Abstract View Class

### Common View Class

In [1464]:
def add_factory_markers():
    return

def add_depot_markers():
    return

def add_customer_markers():
    return

## Base Controller

### Abstract Controller Class

### Common Controller Class

In [1465]:
# HÀM CHÍNH Ở CHỖ CALLBACK Á
# VỚI LẠI, UPDATE CHO A CÁCH TÍNH CÁC COST, A K BIẾT CÁCH TÍNH CÁC COST DO ĐÓ TẠM THỜI ĐỂ GIÁ TRỊ MẶC ĐỊNH = 0. HUYỀN UPDATE CHO A ĐỂ A UPDATE THÔNG TIN NÀY TRONG TOOLTIP!

# Folium Assignment Map Plugin

## Plugin Model
Process Data

In [1466]:
def get_center_of_folium_map(locations_data):
    return None if not locations_data else {
        "lat": sum(loc['lat'] for loc in locations_data) / len(locations_data),
        "lng": sum(loc['lng'] for loc in locations_data) / len(locations_data)
    }

In [1467]:
#11/4
def get_depot_to_pd_mapping(input_data):

    tco_to_physical_mapping = {}

    if "matrixConfig" in input_data and "mapPhysicDepotWithDepot" in input_data["matrixConfig"] and \
            "depots" in input_data and "physicDepots" in input_data:

        depot_to_pd_dict = list_of_dicts_to_plain_dict(input_data["matrixConfig"]["mapPhysicDepotWithDepot"],
                                                       "typeOfDepotByAdress", "typeOfPDByAdress")

        for tco_depot in input_data["depots"]:
            tco_address = tco_depot["depotDetails"]["dType"]["typeOfDepotByAdress"]
            for pd_depot in input_data["physicDepots"]:
                pd_address = pd_depot["physicDepotDetails"]["PDType"]["typeOfPDByAdress"]
                if depot_to_pd_dict.get(tco_address) == pd_address:
                    tco_to_physical_mapping[tco_depot["depotCode"]] = pd_depot["depotPhysicCode"]
                    break  # Break the inner loop once a match is found

    return tco_to_physical_mapping

In [1468]:
def summarize_factory_depot_customer_mapping(tco_to_physical_mapping, output_data):
    # Initialize dictionaries to store the mappings
    factory_to_phy_depot = {}
    factory_to_tco_depot = {}
    phy_depot_to_customer = {}
    tco_depot_to_customer = {}

    if isinstance(output_data, list):
        # Mapping depots to factories and customers
        for depot in output_data:
            if 'depotCode' in depot and 'customers' in depot and 'factories' in depot:
                depot_code = depot['depotCode']
                pd_code = tco_to_physical_mapping.get(depot_code)
                if pd_code is not None:
                    # Mapping depot to customers
                    customers = [customer.get('customerCode') for customer in depot['customers'] if 'customerCode' in customer]
                    tco_depot_to_customer[depot_code] = customers
                    phy_depot_to_customer.setdefault(pd_code, []).extend(customers)
                    # Mapping factories to depot
                    for factory in depot['factories']:
                        if 'factoryCode' in factory:
                            factory_code = factory['factoryCode']
                            # Append the depot code to the respective factory
                            factory_to_tco_depot.setdefault(factory_code, []).append(depot_code)
                            factory_to_phy_depot.setdefault(factory_code, []).append(pd_code)
                            factory_to_phy_depot[factory_code] = list(set(factory_to_phy_depot[factory_code]))

    return {
        'Factory_to_Phy_Depot': factory_to_phy_depot,
        'Factory_to_TCO_Depot': factory_to_tco_depot,
        'Phy_Depot_to_Customer': phy_depot_to_customer,
        'TCO_Depot_to_Customer': tco_depot_to_customer
    }


In [1469]:
tco_to_physical_mapping = get_depot_to_pd_mapping(input_data)

In [1470]:
a = summarize_factory_depot_customer_mapping(tco_to_physical_mapping, output_data)

In [1471]:
a["Factory_to_TCO_Depot"]

{}

In [1472]:
def get_customer_product_capacity(output_data):
    customer_product_capacity = {}

    for depot in output_data["depots"]:
        for customer in depot["customers"]:
            customer_code = customer["customerCode"]

            # Initialize the dictionary for this customer if it does not exist
            if customer_code not in customer_product_capacity:
                customer_product_capacity[customer_code] = {}

            for product in customer["products"]:
                product_code = product["productCode"]
                capacity = product["capacity"]

                # Sum up the capacities for each product code
                if product_code in customer_product_capacity[customer_code]:
                    customer_product_capacity[customer_code][product_code] += capacity
                else:
                    customer_product_capacity[customer_code][product_code] = capacity

    return customer_product_capacity

In [1473]:
def create_assignment_map_data(input_data, output_data, mapping_data, lat_lng_dict):
    assignment_map_data = {
        "factories": {},
        "phys_depots": {},
        "customers": {},
        "mapping_lines": []
    }
    if "matrixConfig" in input_data and "mapPhysicDepotWithFactory" in input_data["matrixConfig"] \
            and "factories" in input_data and "physicDepots" in input_data \
            and "customers" in input_data:

        location_offsets = {}

        pd_in_factory = list_of_dicts_to_plain_dict(input_data["matrixConfig"]["mapPhysicDepotWithFactory"],
                                                    "typeOfFactoryByAdress",
                                                    "typeOfPDByAdress")

        depot_to_pd_mapping = get_depot_to_pd_mapping(input_data)

        pd_to_depot_mapping = reverse_plain_dict_to_dict_of_list(depot_to_pd_mapping)

        cus_prod_capacity = get_customer_product_capacity(output_data)

        factory_to_tco_depot_map = mapping_data.get('Factory_to_TCO_Depot', {})

        for factory in input_data["factories"]:
            if 'factoryCode' in factory and factory['factoryCode'] in lat_lng_dict:
                factory_code = factory["factoryCode"]

                lat = lat_lng_dict[factory_code]["lat"]
                lng = lat_lng_dict[factory_code]["lng"]
                key = (lat, lng)

                location_offsets[key] = location_offsets.get(key, 0) + 1
                offset_index = location_offsets[key] - 1

                offset_lat, offset_lng = add_offset(lat, lng, offset_index) if offset_index > 0 else (lat, lng)
                lat_lng_dict[factory_code]["offset_lat"] = offset_lat
                lat_lng_dict[factory_code]["offset_lng"] = offset_lng

                tco_depots = factory_to_tco_depot_map.get(factory_code, [])

                # Tạo dictionary chứa thông tin các TCO depot
                tcos_info = {}
                for tco_depot_code in tco_depots:
                    tco_depot_info = next((depot for depot in input_data["depots"] if depot["depotCode"] == tco_depot_code), None)

                    if tco_depot_info:
                        handling_out_cost = tco_depot_info.get("handlingOutCost", 0)
                        handling_in_cost = tco_depot_info.get("handlingInCost", 0)
                        ib_transp_cost = calculate_IBtranspCost(input_data["priceByLevelService"], tco_depot_code)
                        ob_transp_cost = calculate_OBtranspCost(input_data["priceByLevelService"], tco_depot_code)

                        tcos_info[tco_depot_code] = {
                            "handlingInCost": handling_in_cost,
                            "handlingOutCost": handling_out_cost,
                            "IBtranspCost": ib_transp_cost,
                            "OBtranspCost": ob_transp_cost
                        }

                assignment_map_data["factories"][factory_code] = {
                    "original_coord": {
                        "lat": lat_lng_dict[factory_code]["lat"],
                        "lng": lat_lng_dict[factory_code]["lng"]
                    },
                    "offset_coord": {
                        "lat": offset_lat,
                        "lng": offset_lng
                    },
                    "factory_penalty_cost": 0,
                    "IB_cost": 0,
                    "min_supply": sum([prod["minVolume"] for prod in factory["products"]]),
                    "max_supply": sum([prod["maxVolume"] for prod in factory["products"]]),
                    "phys_depot_within": pd_in_factory.get(factory_code, "Nothing"),
                    "tcos_info": tcos_info  # Sử dụng dictionary chứa thông tin của TCO depots
                }

                # Trả lại tất cả các TCO depot tương ứng với factory code
                factory_to_tco_depot_map[factory_code] = tco_depots


        for pd in input_data["physicDepots"]:
            if 'depotPhysicCode' in pd and pd['depotPhysicCode'] in pd_to_depot_mapping:
                pd_code = pd["depotPhysicCode"]

                first_depot = pd_to_depot_mapping[pd_code][0]

                lat = lat_lng_dict[first_depot]["lat"]
                lng = lat_lng_dict[first_depot]["lng"]
                key = (lat, lng)

                location_offsets[key] = location_offsets.get(key, 0) + 1
                offset_index = location_offsets[key] - 1

                offset_lat, offset_lng = add_offset(lat, lng, offset_index) if offset_index > 0 else (lat, lng)
                lat_lng_dict[pd_code] = {
                    "lat": lat,
                    "lng": lng,
                    "offset_lat": offset_lat,
                    "offset_lng": offset_lng
                }

                # Find corresponding TCO depot and its handling costs
                tco_depot_info = None
                for tco_depot in input_data["depots"]:
                    if tco_depot["depotCode"] == first_depot:
                        tco_depot_info = tco_depot
                        break

                if tco_depot_info:
                    handling_out_cost = tco_depot_info.get("handlingOutCost", 0)
                    handling_in_cost = tco_depot_info.get("handlingInCost", 0)
                else:
                    handling_out_cost = 0
                    handling_in_cost = 0

                # Tính toán chi phí vận chuyển từ nhà máy đến kho
                ib_transp_cost = calculate_IBtranspCost(input_data["priceByLevelService"], first_depot)

                # Tính toán chi phí vận chuyển từ kho đến khách hàng
                ob_transp_cost = calculate_OBtranspCost(input_data["priceByLevelService"], first_depot)

                assignment_map_data["phys_depots"][pd_code] = {
                    "original_coord": {
                        "lat": lat,
                        "lng": lng
                    },
                    "offset_coord": {
                        "lat": offset_lat,
                        "lng": offset_lng
                    },
                    "tcos_info": {
                        "TCO_depot_code": first_depot,
                        "handlingOutCost": handling_out_cost,
                        "handlingInCost": handling_in_cost,
                        "IBtranspCost": ib_transp_cost,
                        "OBtranspCost" : ob_transp_cost
                    }
                }

        for customer in input_data["customers"]:
            if 'customerCode' in customer and customer['customerCode'] in lat_lng_dict:
                cus_code = customer["customerCode"]
                cus_capacity = cus_prod_capacity.get(cus_code, {})

                lat = lat_lng_dict[cus_code]["lat"]
                lng = lat_lng_dict[cus_code]["lng"]
                key = (lat, lng)

                location_offsets[key] = location_offsets.get(key, 0) + 1
                offset_index = location_offsets[key] - 1

                offset_lat, offset_lng = add_offset(lat, lng, offset_index) if offset_index > 0 else (lat, lng)
                lat_lng_dict[cus_code]["offset_lat"] = offset_lat
                lat_lng_dict[cus_code]["offset_lng"] = offset_lng

                assignment_map_data["customers"][cus_code] = {
                    "original_coord": {
                        "lat": lat_lng_dict[cus_code]["lat"],
                        "lng": lat_lng_dict[cus_code]["lng"]
                    },
                    "offset_coord": {
                        "lat": offset_lat,
                        "lng": offset_lng
                    },
                    "product_info": [
                        {
                            "product_code": product["productCode"],
                            "capacity": cus_capacity.get(product["productCode"], 0),
                            "demand": product.get("meanDemand", 0)
                        } for product in customer["products"] if 'productCode' in product
                    ],
                    "customer_penalty_cost": 0,
                    "OB_cost": 0,
                }


    for factory in mapping_data["Factory_to_Phy_Depot"].keys():
        for phys_depot in mapping_data["Factory_to_Phy_Depot"][factory]:
            if factory in lat_lng_dict and phys_depot in lat_lng_dict:
                map_line = {
                    "type": "factory_to_depot",
                    "start_location": factory,
                    "end_location": phys_depot,
                    "start_coordinate": (lat_lng_dict[factory]["offset_lat"], lat_lng_dict[factory]["offset_lng"]),
                    "end_coordinate": (lat_lng_dict[phys_depot]["offset_lat"], lat_lng_dict[phys_depot]["offset_lng"]),
                    "product_info": []
                }
                assignment_map_data["mapping_lines"].append(map_line)

                map_line = {
                    "type": "depot_to_factory",
                    "start_location": phys_depot,
                    "end_location": factory,
                    "start_coordinate": (lat_lng_dict[phys_depot]["offset_lat"], lat_lng_dict[phys_depot]["offset_lng"]),
                    "end_coordinate": (lat_lng_dict[factory]["offset_lat"], lat_lng_dict[factory]["offset_lng"]),
                    "product_info": []
                }
                assignment_map_data["mapping_lines"].append(map_line)

    for phys_depot in mapping_data["Phy_Depot_to_Customer"]:
        for cus in mapping_data["Phy_Depot_to_Customer"][phys_depot]:
            if phys_depot in lat_lng_dict and cus in lat_lng_dict:
                map_line = {
                    "type": "depot_to_customer",
                    "start_location": phys_depot,
                    "end_location": cus,
                    "start_coordinate": (lat_lng_dict[phys_depot]["offset_lat"], lat_lng_dict[phys_depot]["offset_lng"]),
                    "end_coordinate": (lat_lng_dict[cus]["offset_lat"], lat_lng_dict[cus]["offset_lng"]),
                    "product_info": []
                }
                assignment_map_data["mapping_lines"].append(map_line)

                map_line = {
                    "type": "customer_to_depot",
                    "start_location": cus,
                    "end_location": phys_depot,
                    "start_coordinate": (lat_lng_dict[cus]["offset_lat"], lat_lng_dict[cus]["offset_lng"]),
                    "end_coordinate": (lat_lng_dict[phys_depot]["offset_lat"], lat_lng_dict[phys_depot]["offset_lng"]),
                    "product_info": []
                }
                assignment_map_data["mapping_lines"].append(map_line)

    # Loop through each line in assignment_map_data
    for map_line in assignment_map_data["mapping_lines"]:
        start_location = map_line["start_location"]
        end_location = map_line["end_location"]
        product_info = []  # Initialize list to store product information
  
        # Determine the type of line and process accordingly
        if map_line["type"] == "depot_to_customer":
          
            for customer in output_data["depots"]:
                if customer["depotCode"] == start_location:

                    for customers in customer["customers"]:
                        for product in customers["products"]:
                            product_info.append({
                                "product_code": product["productCode"],
                                "capacity": product["capacity"],  # Set capacity to 0 for depot to customer lines
                            })
                        break
        elif map_line["type"] == "customer_to_depot":

            for customer in output_data["depots"]:
                if customer["depotCode"] == end_location:

                    for customers in customer["customers"]:
                        for product in customers["products"]:
                            product_info.append({
                                "product_code": product["productCode"],
                                "capacity": product["capacity"],  # Set capacity to 0 for depot to customer lines
                            })
                        break
        elif map_line["type"] == "factory_to_depot":

            for depot in output_data["depots"]:
                if depot["depotCode"] == end_location:

                    for factory in depot["factories"]:
                        for product in factory["products"]:
                            product_info.append({
                                "product_code": product["productCode"],
                                "capacity": product["capacity"],  # Set capacity to 0 for depot to customer lines
                            })
                        break
        elif map_line["type"] == "depot_to_factory":

            for depot in output_data["depots"]:
                if depot["depotCode"] == start_location:
    
                    for factory in depot["factories"]:
                        for product in factory["products"]:
                            product_info.append({
                                "product_code": product["productCode"],
                                "capacity": product["capacity"],  # Set capacity to 0 for depot to customer lines
                            })
                        break
        # Assign product information to the current line
        map_line["product_info"] = product_info


    return assignment_map_data



In [1474]:
depot_to_pd_mapping = get_depot_to_pd_mapping(input_data)
pd_to_depot_mapping = reverse_plain_dict_to_dict_of_list(depot_to_pd_mapping)
# pd_to_depot_mapping

## Plugin View
Add Components & Elements to Folium Assignment Map

In [1475]:
MAX_POPUP_BUBBLE_WIDTH = 300

In [1476]:
def assignment_elements_settings():
    return {
        "icon_settings": {
            "factory": {"color": "green", "icon": "industry", "popup_width": MAX_POPUP_BUBBLE_WIDTH},
            "depot": {"color": "red", "icon": "building", "popup_width": MAX_POPUP_BUBBLE_WIDTH},
            "customer": {"color": "blue", "icon": "users", "popup_width": MAX_POPUP_BUBBLE_WIDTH}
        }, "line_settings": {
            "factory_to_depot": {"color": "green"},
            "depot_to_customer": {"color": "red"},
            "customer_to_depot": {"color": "red"},
            "depot_to_factory": {"color": "green"}
        }
    }

In [1477]:
def folium_map_display_settings(map_center):
    return {
        "center": [map_center["lat"], map_center["lng"]],   # Tượng trưng, sẽ load vào sau
        "zoom": 10,         # Có thể thay đổi zoom linh động ở đây
        "display_scalebar": True,
        "tiles": "CartoDB Positron"
    }

In [1478]:
def format_product_info(product_info):

    """
    Tạo chuỗi thông điệp tooltip từ danh sách thông tin sản phẩm.

    Định dạng thông tin về sản phẩm của mỗi khách hàng để hiển thị trên tooltip.
    Mỗi sản phẩm được biểu diễn bởi một dòng, bao gồm mã sản phẩm và yêu cầu sản xuất.

    Input:
        product_info (list of dicts): Danh sách các sản phẩm. Mỗi sản phẩm là
        một dictionary với:
            a. product_code (string): Mã sản phẩm.
            b. demand (int): Sản lượng theo nhu cầu.

    Output:
        string: Một chuỗi HTML, với mỗi sản phẩm trên một dòng mới,
        dạng '- mã sản phẩm: Demand <số lượng demand>'. Mỗi dòng được ngăn cách
        bằng thẻ '<br>' để hiển thị đúng trên tooltip.
    """

    return "<br>".join([f"- {prod['product_code']}: Capacity {prod['capacity']} / Demand {prod['demand']}" for prod in product_info])


In [1479]:
# Toàn bộ các popup content hiện tại cho tượng trưng, sẽ cùng update với Huyền sau

In [1480]:
# def create_factory_popup_content(factory_code, factory_info):
#     penalty_cost = factory_info["factory_penalty_cost"]
#     ib_cost = factory_info["IB_cost"]
#     min_supply = factory_info["min_supply"]
#     max_supply = factory_info["max_supply"]
#     phys_depot_within = factory_info["phys_depot_within"]
#     tcos_info = factory_info["tcos_info"]
#     tco_depot_code = tcos_info["TCO_Depot_Code"]
#     handling_in_cost = tcos_info["handlingInCost"]
#     handling_out_cost = tcos_info["handlingOutCost"]
#     ib_transp_cost =  tcos_info["IBtranspCost"]
#     ob_transp_cost =  tcos_info["OBtranspCost"]

#     return (
#         f"Factory Code: {factory_code}<br>"
#         f"Penalty Cost: {penalty_cost}<br>"
#         f"Inbound Cost: {ib_cost}<br>"
#         f"Min Supply: {min_supply}<br>"
#         f"Max Supply: {max_supply}<br>"
#         f"Physical Depot Within: {phys_depot_within}<br>"
#         f"TCO(s) Depot: {tco_depot_code}<br>"
#         f"- handlingInCost: {handling_in_cost}<br>"
#         f"- handlingOutCost: {handling_out_cost}<br>"
#         f"- IBtranspCost: {ib_transp_cost}<br>"
#         f"- OBtranspCost: {ob_transp_cost}<br>"
#     )

def create_factory_popup_content(factory_code, factory_info):
    penalty_cost = factory_info["factory_penalty_cost"]
    ib_cost = factory_info["IB_cost"]
    min_supply = factory_info["min_supply"]
    max_supply = factory_info["max_supply"]
    phys_depot_within = factory_info["phys_depot_within"]
    tcos_info = factory_info["tcos_info"]

    tco_info_str = ""
    for tco_depot_code, tco in tcos_info.items():
        tco_info_str += (
            f"TCO Depot Code: {tco_depot_code}<br>"
            f"- handlingInCost: {tco['handlingInCost']}<br>"
            f"- handlingOutCost: {tco['handlingOutCost']}<br>"
            f"- IBtranspCost: {tco['IBtranspCost']}<br>"
            f"- OBtranspCost: {tco['OBtranspCost']}<br><br>"
        )

    return (
        f"Factory Code: {factory_code}<br>"
        f"Penalty Cost: {penalty_cost}<br>"
        f"Inbound Cost: {ib_cost}<br>"
        f"Min Supply: {min_supply}<br>"
        f"Max Supply: {max_supply}<br>"
        f"Physical Depot Within: {phys_depot_within}<br>"
        f"{tco_info_str}"
    )

# # Example call to the function (assuming factory_info is already defined)
# factory_code = "some_factory_code"
# factory_info = {
#     "factory_penalty_cost": 0,
#     "IB_cost": 0,
#     "min_supply": 100,
#     "max_supply": 200,
#     "phys_depot_within": "some_depot",
#     "tcos_info": {
#         "TCO1": {
#             "handlingInCost": 10,
#             "handlingOutCost": 20,
#             "IBtranspCost": 30,
#             "OBtranspCost": 40
#         },
#         "TCO2": {
#             "handlingInCost": 15,
#             "handlingOutCost": 25,
#             "IBtranspCost": 35,
#             "OBtranspCost": 45
#         }
#     }
# }
# print(create_factory_popup_content(factory_code, factory_info))


In [1481]:
# def create_phys_depot_popup_content(phys_depot_code, phys_depot_info):
#     tcos_info = phys_depot_info["tcos_info"]
#     tco_depot_code = tcos_info["TCO_depot_code"]
#     handling_in_cost = tcos_info["handlingInCost"]
#     handling_out_cost = tcos_info["handlingOutCost"]

#     return (
#         f"Physical Depot Code: {phys_depot_code}<br>"
#         f"TCO(s) Depot: {tco_depot_code}<br>"
#         f"- handlingInCost: {handling_in_cost}<br>"
#         f"- handlingOutCost: {handling_out_cost}<br>"
#     )
def create_phys_depot_popup_content(phys_depot_code, phys_depot_info):
    tcos_info = phys_depot_info["tcos_info"]
    tco_depot_code = tcos_info["TCO_depot_code"]
    handling_in_cost = tcos_info["handlingInCost"]
    handling_out_cost = tcos_info["handlingOutCost"]
    ib_transp_cost =  tcos_info["IBtranspCost"]
    ob_transp_cost =  tcos_info["OBtranspCost"]

    return (
        f"Physical Depot Code: {phys_depot_code}<br>"
        f"TCO(s) Depot: {tco_depot_code}<br>"
        f"- handlingInCost: {handling_in_cost}<br>"
        f"- handlingOutCost: {handling_out_cost}<br>"
        f"- IBtranspCost: {ib_transp_cost}<br>"
        f"- OBtranspCost: {ob_transp_cost}<br>"
    )


In [1482]:
def create_customer_popup_content(customer_code, customer_info):
    penalty_cost = customer_info["customer_penalty_cost"]
    ob_cost = customer_info["OB_cost"]
    cus_prod_string = format_product_info(customer_info["product_info"])

    return (
        f"Customer Code: {customer_code}<br>"
        f"Product Info:<br>"
        f"{cus_prod_string}<br>"
        f"Penalty Cost: {penalty_cost}<br>"
        f"Outbound Cost: {ob_cost}<br>"
    )

In [1483]:
def create_line_popup_content(line_info):
    start_location = line_info["start_location"]
    end_location = line_info["end_location"]
    line_product = line_info["product_info"]

    # Biến đổi line_product thành chuỗi để hiển thị
    product_info_str = "<br>".join([f"{product['productCode']}: {product['capacity']} (Inbound Cost: {product.get('inboundCost', 0)}, Outbound Cost: {product.get('outboundCost', 0)})" for product in line_product])

    return (
        f"Pickup Location Code: {start_location}<br>"
        f"Delivery Location Code: {end_location}<br>"
        f"Product Info:<br>{product_info_str}<br>"
    )


In [1484]:
#11/4
def create_markers_for_assignment_map(assignment_map_data, icon_settings):
    markers_array = []

    if "factories" in assignment_map_data:
        for fac, fac_info in assignment_map_data["factories"].items():
            if "offset_coord" in fac_info:
                fac_marker_info = {}
                fac_marker_info["color"] = icon_settings.get("factory", {}).get("color", "blue")
                fac_marker_info["icon"] = icon_settings.get("factory", {}).get("icon", "default_factory_icon")
                fac_marker_info["fill"] = True
                fac_marker_info["radius"] = 500
                fac_marker_info["location"] = [fac_info["offset_coord"].get("lat", 0), fac_info["offset_coord"].get("lng", 0)]
                fac_marker_info["popup"] = {
                    "content": create_factory_popup_content(fac, fac_info),
                    "width": MAX_POPUP_BUBBLE_WIDTH
                }

                fac_marker = create_plain_marker_element(fac_marker_info)

                markers_array.append(fac_marker)

    if "phys_depots" in assignment_map_data:
        for pd, pd_info in assignment_map_data["phys_depots"].items():
            if "offset_coord" in pd_info:
                pd_marker_info = {}
                pd_marker_info["color"] = icon_settings.get("depot", {}).get("color", "green")
                pd_marker_info["icon"] = icon_settings.get("depot", {}).get("icon", "default_depot_icon")
                pd_marker_info["fill"] = True
                pd_marker_info["radius"] = 500
                pd_marker_info["location"] = [pd_info["offset_coord"].get("lat", 0), pd_info["offset_coord"].get("lng", 0)]
                pd_marker_info["popup"] = {
                    "content": create_phys_depot_popup_content(pd, pd_info),
                    "width": MAX_POPUP_BUBBLE_WIDTH
                }

                pd_marker = create_plain_marker_element(pd_marker_info)

                markers_array.append(pd_marker)

    if "customers" in assignment_map_data:
        for cus, cus_info in assignment_map_data["customers"].items():
            if "offset_coord" in cus_info:
                cus_marker_info = {}
                cus_marker_info["color"] = icon_settings.get("customer", {}).get("color", "red")
                cus_marker_info["icon"] = icon_settings.get("customer", {}).get("icon", "default_customer_icon")
                cus_marker_info["fill"] = True
                cus_marker_info["radius"] = 200
                cus_marker_info["location"] = [cus_info["offset_coord"].get("lat", 0), cus_info["offset_coord"].get("lng", 0)]
                cus_marker_info["popup"] = {
                    "content": create_customer_popup_content(cus, cus_info),
                    "width": MAX_POPUP_BUBBLE_WIDTH
                }

                cus_marker = create_plain_marker_element(cus_marker_info)

                markers_array.append(cus_marker)

    return markers_array


In [1485]:
# #11/4
# def create_lines_for_assignment_map(assignment_map_data, line_settings, selected_locations=None):
#     lines_array = []

#     if "mapping_lines" in assignment_map_data:
#         for line in assignment_map_data["mapping_lines"]:
#             if "start_location" in line and "end_location" in line and "type" in line \
#                     and "start_coordinate" in line and "end_coordinate" in line:
#                 start_location = line["start_location"]
#                 end_location = line["end_location"]

#                 # Draw all lines by default
#                 draw_line = True

#                 # Check if selected locations are provided
#                 if selected_locations:
#                     # Filter lines based on selected locations
#                     if start_location not in selected_locations or end_location not in selected_locations:
#                         draw_line = False

#                 if draw_line:
#                     line_type = line["type"]

#                     line_info = {}
#                     line_info["line_color"] = line_settings.get(line_type, {}).get("color", "black")
#                     line_info["start"] = line["start_coordinate"]
#                     line_info["end"] = line["end_coordinate"]
#                     line_info["popup"] = {
#                         "content": create_line_popup_content(line),
#                         "width": MAX_POPUP_BUBBLE_WIDTH
#                     }
#                     line_info["arrow_sides"] = 3
#                     line_info["arrow_radius"] = 5
#                     line_info["weight"] = 3

#                     polyline = create_arrowed_line_element(line_info)

#                     lines_array.append(polyline)

#     return lines_array
def create_lines_for_assignment_map(assignment_map_data, line_settings, selected_locations=None):
    lines_array = []

    if "mapping_lines" in assignment_map_data:
        for line in assignment_map_data["mapping_lines"]:
            if "start_location" in line and "end_location" in line and "type" in line \
                    and "start_coordinate" in line and "end_coordinate" in line:
                start_location = line["start_location"]
                end_location = line["end_location"]

                draw_line = True

                if selected_locations:
                    if start_location not in selected_locations or end_location not in selected_locations:
                        draw_line = False

                if draw_line:
                    line_type = line["type"]

                    # Lấy màu từ line_settings, mặc định là đen nếu không được xác định
                    line_color = line_settings.get(line_type, {}).get("color", "black")

                    line_info = {
                        "line_color": line_color,
                        "start": line["start_coordinate"],
                        "end": line["end_coordinate"],
                        "popup": {
                            "content": create_line_popup_content(line),
                            "width": MAX_POPUP_BUBBLE_WIDTH
                        },
                        "arrow_sides": 3,
                        "arrow_radius": 5,
                        "weight": 3
                    }

                    polyline = create_arrowed_line_element(line_info)

                    lines_array.append(polyline)

    return lines_array


In [1486]:
def create_legend_text_to_folium_map(input_data, output_data):
    ib_transport = output_data["IB_transport"]
    ob_transport = output_data["OB_transport"]
    handling_cost = output_data["handling_cost"]
    depot_cost = output_data["cost_depot"]
    total_cost = output_data["cost_all"]
    cus_pen_cost = output_data["cost_penalty_customer"]
    fac_pen_cost = output_data["cost_penalty_factory"]
    used_depots = len(output_data["useDepot"])
    total_p_deps = len(input_data["physicDepots"])
    total_t_deps = len(output_data["depots"])

    # Create the legend HTML for the Folium Map
    legend_html = f'''
    <div style="position: fixed;
        bottom: 20px; right: 20px; width: 250px; height: 250px;
        border:2px solid grey; z-index:9999; font-size:14px;
        background-color:yellow; padding:5px;">
        <strong>Result Summary</strong><br>
        IB Transport: {ib_transport}<br>
        OB Transport: {ob_transport}<br>
        Handling Cost: {handling_cost}<br>
        Depot Cost: {depot_cost}<br>
        Total Cost: {total_cost}<br>
        Factory Penalty Cost: {fac_pen_cost}<br>
        Customer Penalty Cost: {cus_pen_cost}<br>
        Number of Used (Physical) Depots: {used_depots}<br>
        Total Physical Depots: {total_p_deps}<br>
        Total TCO Depots: {total_t_deps}<br>
    </div>
    '''

    return legend_html

In [1487]:
def get_visualization_2(map_settings, markers, lines, legend_html):
    return generate_folium_assignment_map(map_settings, markers, lines, legend_html)

## Plugin Controller
Create Folium Assignment Map

In [1488]:
def create_visualization_2(input_data, output_data, has_line, selected_locations):
    # Initialize or update lat_lng_dict with default values if necessary
    lat_lng_dict = {}  # Initialize the dictionary
    if "locations" in input_data and isinstance(input_data["locations"], list):
        for phys_depot in input_data["locations"]:
            lat_lng_dict[phys_depot["locationCode"]] = {
                'lat': phys_depot.get("lat", 0),
                'lng': phys_depot.get("lng", 0),
                'offset_lat': phys_depot.get("lat", 0),
                'offset_lng': phys_depot.get("lng", 0)
            }

            if "offset_lat" not in lat_lng_dict[phys_depot["locationCode"]]:
                lat_lng_dict[phys_depot["locationCode"]]["offset_lat"] = 0  # or any default value you want
            if "offset_lng" not in lat_lng_dict[phys_depot["locationCode"]]:
                lat_lng_dict[phys_depot["locationCode"]]["offset_lng"] = 0  # or any default value you want

    # Your existing code follows...
    map_center = get_center_of_folium_map(input_data["locations"]) if "locations" in input_data else (0, 0) # Default to (0, 0) if no locations
    tco_to_physical_mapping = get_depot_to_pd_mapping(input_data)
    result_mapping_data = summarize_factory_depot_customer_mapping(tco_to_physical_mapping, output_data["depots"])
    assignment_map_data = create_assignment_map_data(input_data, output_data, result_mapping_data, lat_lng_dict)

    elements_settings = assignment_elements_settings()
    display_settings = folium_map_display_settings(map_center)
    legend_html = create_legend_text_to_folium_map(input_data, output_data)
    folium_markers = create_markers_for_assignment_map(assignment_map_data, elements_settings["icon_settings"])
    folium_lines = create_lines_for_assignment_map(assignment_map_data, elements_settings["line_settings"], selected_locations) if has_line else []
    folium_map = get_visualization_2(display_settings, folium_markers, folium_lines, legend_html)

    return folium_map


## Component Helper Functions

In [1489]:
def add_markers_to_map(folium_map, markers_array):
    for marker in markers_array:
        marker.add_to(folium_map)

In [1490]:
# Viết code tổng quát nhất có thể, add line & các decorators của line đi kèm
def add_lines_to_map(folium_map, lines_array):
    for line in lines_array:
        for part in line:
            line[part].add_to(folium_map)
        # line["line"].add_to(folium_map)
        # if "arrow" in line.keys():
        #     line["arrow"].add_to(folium_map)

In [1491]:
def add_legend_to_map(folium_map, legend_content):
    # Create a template with the legend HTML
    template = Template(f"""
        {{% macro html(this, kwargs) %}}
        {legend_content}
        {{% endmacro %}}
    """)

    # Create a macro element to attach the template to the map
    macro = MacroElement()
    macro._template = template

    # Add the macro (and therefore the legend) to the map
    folium_map.get_root().add_child(macro)

## Component
Assignment Map

In [1492]:
def generate_folium_assignment_map(map_settings, markers, lines, legend_content):

    fmap = folium.Map(location=map_settings["center"], zoom_start=map_settings["zoom"],
                      control_scale=map_settings["display_scalebar"])

    add_markers_to_map(fmap, markers)

    add_lines_to_map(fmap, lines)

    add_legend_to_map(fmap, legend_content)

    return fmap

## Elements

### Element Helper Functions

In [1493]:
# Chỉnh cái này ở constants
backward_distance = 4

In [1494]:
# https://pyproj4.github.io/pyproj/stable/api/geod.html
from pyproj import Geod

# Initialize Geod
geod = Geod(ellps='clrk66')

def calculate_azimuth(line_info):
    azimuth, _, _ = geod.inv(line_info["start"][1], line_info["start"][0],
                             line_info["end"][1], line_info["end"][0])
    return azimuth

def calculate_new_coord(line_info, azimuth):
    new_lng, new_lat, _ = geod.fwd(line_info["end"][1], line_info["end"][0],
                                   azimuth + 180, backward_distance)
    return new_lng, new_lat

### Arrowed Line

In [1495]:
def create_arrowed_line_element(line_info):
    line_display = {}

    popup_bubble = folium.Popup(line_info["popup"]["content"],
                                max_width=line_info["popup"]["width"])

    line_display["line"] = folium.PolyLine([line_info["start"], line_info["end"]],
                                           color=line_info["line_color"],
                                           weight=line_info["weight"],
                                           popup=popup_bubble)

    azimuth = calculate_azimuth(line_info)

    new_lng, new_lat = calculate_new_coord(line_info, azimuth)

    line_display["arrow"] = folium.RegularPolygonMarker(
        (new_lat, new_lng),
        fill_color=line_info["line_color"],
        number_of_sides=line_info["arrow_sides"],
        radius=line_info["arrow_radius"],
        rotation=azimuth-90
    )

    return line_display

### Marker

In [1496]:
def create_circle_marker_element(marker_info):

    popup_bubble = folium.Popup(marker_info["popup"]["content"],
                                max_width=marker_info["popup"]["width"])

    return folium.Circle(
        location=marker_info["location"],
        color=marker_info["color"],
        fill=marker_info["fill"],
        fill_color=marker_info["color"],
        popup=popup_bubble,
        radius=marker_info["radius"]
    )

In [1497]:
def create_plain_marker_element(marker_info):

    popup_bubble = folium.Popup(marker_info["popup"]["content"],
                            max_width=marker_info["popup"]["width"])

    return folium.Marker(
        location=marker_info["location"],
        icon=folium.Icon(color=marker_info["color"], prefix = 'fa', icon=marker_info["icon"]),
        popup=popup_bubble
    )

# Scatter Assigment Map Plugin

## Plugin Model
Process Data

In [1498]:
# get_center_of_folium_map
def get_scatter_map_center(locations_data):
    return None if not locations_data else {
        "lat": sum(loc['lat'] for loc in locations_data) / len(locations_data),
        "lng": sum(loc['lng'] for loc in locations_data) / len(locations_data)
    }

In [1499]:
#11/4
def get_depot_to_pd_mapping_sm(input_data):

    tco_to_physical_mapping = {}

    if "matrixConfig" in input_data and "mapPhysicDepotWithDepot" in input_data["matrixConfig"] and \
            "depots" in input_data and "physicDepots" in input_data:

        depot_to_pd_dict = list_of_dicts_to_plain_dict(input_data["matrixConfig"]["mapPhysicDepotWithDepot"],
                                                       "typeOfDepotByAdress", "typeOfPDByAdress")

        for tco_depot in input_data["depots"]:
            tco_address = tco_depot["depotDetails"]["dType"]["typeOfDepotByAdress"]
            for pd_depot in input_data["physicDepots"]:
                pd_address = pd_depot["physicDepotDetails"]["PDType"]["typeOfPDByAdress"]
                if depot_to_pd_dict.get(tco_address) == pd_address:
                    tco_to_physical_mapping[tco_depot["depotCode"]] = pd_depot["depotPhysicCode"]
                    break  # Break the inner loop once a match is found

    return tco_to_physical_mapping


In [1500]:
def summarize_factory_depot_customer_mapping_sm(tco_to_physical_mapping, output_data):
    # Initialize dictionaries to store the mappings
    factory_to_phy_depot = {}
    factory_to_tco_depot = {}
    phy_depot_to_customer = {}
    tco_depot_to_customer = {}

    phy_depot_to_factory = {}
    customer_to_phy_depot = {}

    if isinstance(output_data, list):
        # Mapping depots to factories and customers
        for depot in output_data:
            if 'depotCode' in depot and 'customers' in depot and 'factories' in depot:
                depot_code = depot['depotCode']
                pd_code = tco_to_physical_mapping.get(depot_code)
                if pd_code is not None:
                    # Mapping depot to customers
                    customers = [customer.get('customerCode') for customer in depot['customers'] if 'customerCode' in customer]
                    tco_depot_to_customer[depot_code] = customers
                    phy_depot_to_customer.setdefault(pd_code, []).extend(customers)
                    # Mapping factories to depot
                    for factory in depot['factories']:
                        if 'factoryCode' in factory:
                            factory_code = factory['factoryCode']
                            # Append the depot code to the respective factory
                            factory_to_tco_depot.setdefault(factory_code, []).append(depot_code)
                            factory_to_phy_depot.setdefault(factory_code, []).append(pd_code)
                            factory_to_phy_depot[factory_code] = list(set(factory_to_phy_depot[factory_code]))

        for factory, depots in factory_to_phy_depot.items():
            for depot in depots:
                if depot not in phy_depot_to_factory:
                    phy_depot_to_factory[depot] = [factory]
                else:
                    phy_depot_to_factory[depot].append(factory)

        for depot, customers in phy_depot_to_customer.items():
            for customer in customers:
                if customer not in customer_to_phy_depot:
                    customer_to_phy_depot[customer] = [depot]
                else:
                    customer_to_phy_depot[customer].append(depot)

    return {
        'Factory_to_Phy_Depot': factory_to_phy_depot,
        'Factory_to_TCO_Depot': factory_to_tco_depot,
        'Phy_Depot_to_Customer': phy_depot_to_customer,
        'TCO_Depot_to_Customer': tco_depot_to_customer,
        'Customer_to_Phy_Depot': customer_to_phy_depot,
        'Phy_Depot_to_Factory': phy_depot_to_factory,
    }


In [1501]:
def get_customer_product_capacity_sm(output_data):
    customer_product_capacity = {}

    for depot in output_data["depots"]:
        for customer in depot["customers"]:
            customer_code = customer["customerCode"]

            # Initialize the dictionary for this customer if it does not exist
            if customer_code not in customer_product_capacity:
                customer_product_capacity[customer_code] = {}

            for product in customer["products"]:
                product_code = product["productCode"]
                capacity = product["capacity"]

                # Sum up the capacities for each product code
                if product_code in customer_product_capacity[customer_code]:
                    customer_product_capacity[customer_code][product_code] += capacity
                else:
                    customer_product_capacity[customer_code][product_code] = capacity

    return customer_product_capacity

In [1502]:
#11/4
def create_assignment_map_data_sm(input_data, output_data, mapping_data, lat_lng_dict):
    assignment_map_data = {
        "factories": {},
        "phys_depots": {},
        "customers": {},
        "mapping_lines": []
    }

    if "matrixConfig" in input_data and "mapPhysicDepotWithFactory" in input_data["matrixConfig"] \
            and "factories" in input_data and "physicDepots" in input_data \
            and "customers" in input_data:

        location_offsets = {}

        pd_in_factory = list_of_dicts_to_plain_dict(input_data["matrixConfig"]["mapPhysicDepotWithFactory"],
                                                    "typeOfFactoryByAdress",
                                                    "typeOfPDByAdress")

        depot_to_pd_mapping = get_depot_to_pd_mapping_sm(input_data)

        pd_to_depot_mapping = reverse_plain_dict_to_dict_of_list(depot_to_pd_mapping)

        cus_prod_capacity = get_customer_product_capacity_sm(output_data)

        for factory in input_data["factories"]:
            if 'factoryCode' in factory and factory['factoryCode'] in lat_lng_dict:
                factory_code = factory["factoryCode"]

                lat = lat_lng_dict[factory_code]["lat"]
                lng = lat_lng_dict[factory_code]["lng"]
                key = (lat, lng)

                location_offsets[key] = location_offsets.get(key, 0) + 1
                offset_index = location_offsets[key] - 1

                offset_lat, offset_lng = add_offset(lat, lng, offset_index) if offset_index > 0 else (lat, lng)
                lat_lng_dict[factory_code]["offset_lat"] = offset_lat
                lat_lng_dict[factory_code]["offset_lng"] = offset_lng

                assignment_map_data["factories"][factory_code] = {
                    "original_coord": {
                        "lat": lat_lng_dict[factory_code]["lat"],
                        "lng": lat_lng_dict[factory_code]["lng"]
                    },
                    "offset_coord": {
                        "lat": offset_lat,
                        "lng": offset_lng
                    },
                    "factory_penalty_cost": 0,      # Chi phí tượng trưng
                    "IB_cost": 0,                   # Chi phí tượng trưng
                    "min_supply": sum([prod["minVolume"] for prod in factory["products"]]),
                    "max_supply": sum([prod["maxVolume"] for prod in factory["products"]]),
                    "phys_depot_within": pd_in_factory.get(factory_code, "Nothing"),
                    "tcos_info": "Pending TCOs Information"     # Sẽ thêm chi tiết vào sau
                }

        for pd in input_data["physicDepots"]:
            if 'depotPhysicCode' in pd and pd['depotPhysicCode'] in pd_to_depot_mapping:
                pd_code = pd["depotPhysicCode"]

                first_depot = pd_to_depot_mapping[pd_code][0]

                lat = lat_lng_dict[first_depot]["lat"]
                lng = lat_lng_dict[first_depot]["lng"]
                key = (lat, lng)

                location_offsets[key] = location_offsets.get(key, 0) + 1
                offset_index = location_offsets[key] - 1

                offset_lat, offset_lng = add_offset(lat, lng, offset_index) if offset_index > 0 else (lat, lng)
                lat_lng_dict[pd_code] = {
                    "lat": lat,
                    "lng": lng,
                    "offset_lat": offset_lat,
                    "offset_lng": offset_lng
                }

                assignment_map_data["phys_depots"][pd_code] = {
                    "original_coord": {
                        "lat": lat,
                        "lng": lng
                    },
                    "offset_coord": {
                        "lat": offset_lat,
                        "lng": offset_lng
                    },
                    "tcos_info": "Pending TCOs Information"
                }

        for customer in input_data["customers"]:
            if 'customerCode' in customer and customer['customerCode'] in lat_lng_dict:
                cus_code = customer["customerCode"]
                cus_capacity = cus_prod_capacity.get(cus_code, {})

                lat = lat_lng_dict[cus_code]["lat"]
                lng = lat_lng_dict[cus_code]["lng"]
                key = (lat, lng)

                location_offsets[key] = location_offsets.get(key, 0) + 1
                offset_index = location_offsets[key] - 1

                offset_lat, offset_lng = add_offset(lat, lng, offset_index) if offset_index > 0 else (lat, lng)
                lat_lng_dict[cus_code]["offset_lat"] = offset_lat
                lat_lng_dict[cus_code]["offset_lng"] = offset_lng

                assignment_map_data["customers"][cus_code] = {
                    "original_coord": {
                        "lat": lat_lng_dict[cus_code]["lat"],
                        "lng": lat_lng_dict[cus_code]["lng"]
                    },
                    "offset_coord": {
                        "lat": offset_lat,
                        "lng": offset_lng
                    },
                    "product_info": [
                        {
                            "product_code": product["productCode"],
                            "capacity": cus_capacity.get(product["productCode"], 0),
                            "demand": product.get("meanDemand", 0)
                        } for product in customer["products"] if 'productCode' in product
                    ],
                    "customer_penalty_cost": 0,     # Chi phí tượng trưng
                    "OB_cost": 0,                   # Chi phí tượng trưng
                }

        for factory in mapping_data["Factory_to_Phy_Depot"].keys():
            for phys_depot in mapping_data["Factory_to_Phy_Depot"][factory]:
                if factory in lat_lng_dict and phys_depot in lat_lng_dict:
                    map_line = {
                        "type": "factory_to_phy_depot",
                        "start_location": factory,
                        "end_location": phys_depot,
                        "start_coordinate": (lat_lng_dict[factory]["offset_lat"], lat_lng_dict[factory]["offset_lng"]),
                        "end_coordinate": (lat_lng_dict[phys_depot]["offset_lat"], lat_lng_dict[phys_depot]["offset_lng"]),
                        # "product_info": "Pending Product Information"       # Sẽ thêm chi tiết vào sau
                    }
                    assignment_map_data["mapping_lines"].append(map_line)

        for phys_depot in mapping_data["Phy_Depot_to_Customer"]:
            for cus in mapping_data["Phy_Depot_to_Customer"][phys_depot]:
                if phys_depot in lat_lng_dict and cus in lat_lng_dict:
                    map_line = {
                        "type": "phy_depot_to_customer",
                        "start_location": phys_depot,
                        "end_location": cus,
                        "start_coordinate": (lat_lng_dict[phys_depot]["offset_lat"], lat_lng_dict[phys_depot]["offset_lng"]),
                        "end_coordinate": (lat_lng_dict[cus]["offset_lat"], lat_lng_dict[cus]["offset_lng"]),
                        # "product_info": "Pending Product Information"       # Sẽ thêm chi tiết vào sau
                    }
                    assignment_map_data["mapping_lines"].append(map_line)

        for cus in mapping_data["Customer_to_Phy_Depot"]:  # Iterate over physical depots
            for phys_depot in mapping_data["Customer_to_Phy_Depot"][cus]:  # Iterate over customers in each physical depot
                if phys_depot in lat_lng_dict and cus in lat_lng_dict:  # Check if both locations have coordinates
                    # Create mapping line object
                    map_line = {
                        "type": "customer_to_phy_depot",  # Changed type to reflect the direction
                        "start_location": cus,  # Start from customer
                        "end_location": phys_depot,  # End at physical depot
                        "start_coordinate": (lat_lng_dict[cus]["offset_lat"], lat_lng_dict[cus]["offset_lng"]),
                        "end_coordinate": (lat_lng_dict[phys_depot]["offset_lat"], lat_lng_dict[phys_depot]["offset_lng"]),  # Swap coordinates
                        # "product_info": "Pending Product Information"  # Placeholder for product info
                    }
                    assignment_map_data["mapping_lines"].append(map_line)  # Add mapping line to the list

        for phy_depot in mapping_data["Phy_Depot_to_Factory"].keys():  # Iterate over physical depots
            for factory in mapping_data["Phy_Depot_to_Factory"][phy_depot]:  # Iterate over factories in each physical depot
                if factory in lat_lng_dict and phy_depot in lat_lng_dict:  # Check if both locations have coordinates
                    # Create mapping line object
                    map_line = {
                        "type": "phy_depot_to_factory",  # Changed type to reflect the direction
                        "start_location": phy_depot,  # Start from physical depot
                        "end_location": factory,  # End at factory
                        "start_coordinate": (lat_lng_dict[phy_depot]["offset_lat"], lat_lng_dict[phy_depot]["offset_lng"]),
                        "end_coordinate": (lat_lng_dict[factory]["offset_lat"], lat_lng_dict[factory]["offset_lng"]),  # Swap coordinates
                        # "product_info": "Pending Product Information"  # Placeholder for product info
                    }
                    assignment_map_data["mapping_lines"].append(map_line)  # Add mapping line to the list


    return assignment_map_data


## Plugin View

In [1503]:
MAX_POPUP_BUBBLE_WIDTH = 300

In [1504]:
def create_line_popup_content_sm(line_info):
    start_location = line_info["start_location"]
    end_location = line_info["end_location"]

    return (
        f"Pickup Location Code: {start_location}<br>"
        f"Delivery Location Code: {end_location}<br>"
        f"Demand:<br>"
        f"    - SKU-A: Unknown<br>"
        f"    - SKU-B: Unknown<br>"
    )

In [1505]:
backward_distance = 4

In [1506]:
#11/4
def create_lines_for_scatter_assignment_map(assignment_map_data, line_settings, selected_locations=None):
    lines_array = []

    if "mapping_lines" in assignment_map_data:
        for line in assignment_map_data["mapping_lines"]:
            if "start_location" in line and "end_location" in line and "type" in line \
                    and "start_coordinate" in line and "end_coordinate" in line:
                start_location = line["start_location"]
                end_location = line["end_location"]

                # Draw all lines by default
                draw_line = True

                # Check if selected locations are provided
                if selected_locations:
                    # Filter lines based on selected locations
                    if start_location not in selected_locations or end_location not in selected_locations:
                        draw_line = False

                if draw_line:
                    line_type = line["type"]

                    line_info = {}
                    line_info["line_color"] = line_settings.get(line_type, {}).get("color", "black")
                    line_info["start"] = line["start_coordinate"]
                    line_info["end"] = line["end_coordinate"]
                    line_info["popup"] = {
                        "content": create_line_popup_content_sm(line),
                        "width": MAX_POPUP_BUBBLE_WIDTH
                    }
                    line_info["arrow_sides"] = 3
                    line_info["arrow_radius"] = 5
                    line_info["weight"] = 3

                    # polyline = create_arrowed_line_element(line_info)

                    # lines_array.append(polyline)

    return lines_array


In [1507]:
mapbox_config = {
    'accesstoken': 'pk.eyJ1Ijoic29sa2FyYW1tYWFyIiwiYSI6ImNrc2JqdWM4eTA3aXYzMG9kOGd2NTZxdHEifQ.M8vfidWZ-3C7SEI2w4EjLg',
    'style': 'mapbox://styles/mapbox/streets-v11',
    'zoom': 8,
    'center': {'lat': 10.7636, 'lon': 106.6602}  # Tọa độ trung tâm bản đồ
}

In [1508]:
def create_fig():
    # Initial plot
    fig = go.Figure()

    # Thêm các smaller traces
    for location_type, color in zip(['factories', 'customers', 'phys_depots'], ['green', 'blue', 'red']):
        for loc_code, loc_info in assignment_map_data[location_type].items():
            fig.add_trace(
                go.Scattermapbox(
                    lat=[loc_info['offset_coord']['lat']],
                    lon=[loc_info['offset_coord']['lng']],
                    text=loc_code,
                    hoverinfo='text',
                    mode='markers',
                    marker=dict(size=10, color=color),
                    name=location_type.capitalize()  # Capitalize the location type for legend
                )
            )
    fig.update_layout(
        title='Interactive Point and Line Visibility',
        clickmode='event+select',
        hovermode='closest',
        mapbox=mapbox_config,
        showlegend=False
    )
    return fig

## Plugin Controller

# Put into Dash App

### Model dash

In [1509]:
folium_map = create_visualization_2(input_data, output_data, True, None)
# folium_map

In [1510]:
lat_lng_dict = list_of_dicts_to_dict_of_dicts(input_data["locations"], "locationCode")

In [1511]:
map_center = get_center_of_folium_map(input_data["locations"])
map_center

{'lat': 10.330221891817796, 'lng': 106.0770745470339}

In [1512]:
tco_to_physical_mapping = get_depot_to_pd_mapping(input_data)
# tco_to_physical_mapping

In [1513]:
result_mapping_data = summarize_factory_depot_customer_mapping(tco_to_physical_mapping, output_data["depots"])


In [1514]:
result_mapping_data.keys()

dict_keys(['Factory_to_Phy_Depot', 'Factory_to_TCO_Depot', 'Phy_Depot_to_Customer', 'TCO_Depot_to_Customer'])

In [1515]:
result_mapping_data['Factory_to_TCO_Depot']

{'061': ['074',
  '070',
  '075',
  'DC165',
  'DC146',
  'DC061',
  'DC160',
  'DC150',
  'DC162'],
 '162': ['074',
  '070',
  '086',
  '080A',
  '060A',
  '064B',
  'DC165',
  'DC146',
  'DCCC',
  'DC145',
  'DC162'],
 '165': ['074', 'DC165'],
 'CC': ['074',
  '070',
  '075',
  '086',
  '080A',
  '060A',
  '064B',
  'DC153',
  'DC165',
  'DC146',
  'DCCC',
  'DC163',
  'DC145',
  'DC160',
  'DC150',
  'DC162'],
 '163': ['070', '075', '080A', 'DC153', 'DC163', 'DC160', 'DC150'],
 '166': ['070', '075', 'DC146', 'DC166', 'DC162'],
 '153': ['086', 'DC153', 'DC160'],
 '160': ['080A', 'DC153', 'DC160', 'DC150'],
 '146': ['060A', 'DC146'],
 '27': ['DCCC'],
 '145': ['DC145'],
 '150': ['DC150']}

In [1516]:
result_mapping_data['Factory_to_Phy_Depot']

{'061': ['PD-162',
  'PD-160',
  'PD-146',
  'PD-075',
  'PD-061',
  'PD-150',
  'PD-165',
  'PD-074',
  'PD-070'],
 '162': ['PD-CC',
  'PD-162',
  'PD-146',
  'PD-064B',
  'PD-086',
  'PD-060A',
  'PD-165',
  'PD-074',
  'PD-070',
  'PD-145',
  'PD-080A'],
 '165': ['PD-165', 'PD-074'],
 'CC': ['PD-CC',
  'PD-162',
  'PD-153',
  'PD-160',
  'PD-146',
  'PD-163',
  'PD-064B',
  'PD-075',
  'PD-086',
  'PD-150',
  'PD-060A',
  'PD-165',
  'PD-074',
  'PD-070',
  'PD-145',
  'PD-080A'],
 '163': ['PD-153',
  'PD-160',
  'PD-163',
  'PD-075',
  'PD-150',
  'PD-070',
  'PD-080A'],
 '166': ['PD-162', 'PD-146', 'PD-166', 'PD-075', 'PD-070'],
 '153': ['PD-086', 'PD-153', 'PD-160'],
 '160': ['PD-080A', 'PD-153', 'PD-160', 'PD-150'],
 '146': ['PD-060A', 'PD-146'],
 '27': ['PD-CC'],
 '145': ['PD-145'],
 '150': ['PD-150']}

In [1517]:
result_mapping_data['Phy_Depot_to_Factory'] = {}

for factory, depots in result_mapping_data['Factory_to_Phy_Depot'].items():
    for depot in depots:
        if depot not in result_mapping_data['Phy_Depot_to_Factory']:
            result_mapping_data['Phy_Depot_to_Factory'][depot] = [factory]
        else:
            result_mapping_data['Phy_Depot_to_Factory'][depot].append(factory)
result_mapping_data['Phy_Depot_to_Factory']

{'PD-162': ['061', '162', 'CC', '166'],
 'PD-160': ['061', 'CC', '163', '153', '160'],
 'PD-146': ['061', '162', 'CC', '166', '146'],
 'PD-075': ['061', 'CC', '163', '166'],
 'PD-061': ['061'],
 'PD-150': ['061', 'CC', '163', '160', '150'],
 'PD-165': ['061', '162', '165', 'CC'],
 'PD-074': ['061', '162', '165', 'CC'],
 'PD-070': ['061', '162', 'CC', '163', '166'],
 'PD-CC': ['162', 'CC', '27'],
 'PD-064B': ['162', 'CC'],
 'PD-086': ['162', 'CC', '153'],
 'PD-060A': ['162', 'CC', '146'],
 'PD-145': ['162', 'CC', '145'],
 'PD-080A': ['162', 'CC', '163', '160'],
 'PD-153': ['CC', '163', '153', '160'],
 'PD-163': ['CC', '163'],
 'PD-166': ['166']}

In [1518]:
# result_mapping_data['Phy_Depot_to_Customer']

In [1519]:
result_mapping_data['Customer_to_Phy_Depot'] = {}

for depot, customers in result_mapping_data['Phy_Depot_to_Customer'].items():
    for customer in customers:
        if customer not in result_mapping_data['Customer_to_Phy_Depot']:
            result_mapping_data['Customer_to_Phy_Depot'][customer] = [depot]
        else:
            result_mapping_data['Customer_to_Phy_Depot'][customer].append(depot)
result_mapping_data['Customer_to_Phy_Depot']

{'CIZ0100-0778': ['PD-074', 'PD-165', 'PD-CC'],
 'CIZ1905-0004': ['PD-074', 'PD-165', 'PD-061', 'PD-CC', 'PD-162'],
 'CIZ1905-0005': ['PD-074', 'PD-CC'],
 'CIZ1905-0006': ['PD-074', 'PD-070', 'PD-165', 'PD-061', 'PD-CC'],
 'CIZ1905-0009': ['PD-074', 'PD-CC'],
 'CIZ1905-0010': ['PD-074', 'PD-153', 'PD-061', 'PD-CC', 'PD-160', 'PD-162'],
 'CIZ1905-0012': ['PD-074', 'PD-CC'],
 'CIZ1905-0014': ['PD-074', 'PD-153', 'PD-061', 'PD-CC', 'PD-160', 'PD-162'],
 'CIZ1905-0016': ['PD-074', 'PD-153', 'PD-061', 'PD-CC', 'PD-160', 'PD-162'],
 'CIZ1905-0056': ['PD-074', 'PD-153', 'PD-CC', 'PD-160', 'PD-150', 'PD-162'],
 'CIZ1905-0079': ['PD-074', 'PD-153', 'PD-061', 'PD-CC', 'PD-160', 'PD-162'],
 'CIZ2001-0001': ['PD-074', 'PD-153', 'PD-CC', 'PD-163', 'PD-160'],
 'CIZ2001-0120': ['PD-074', 'PD-153', 'PD-CC', 'PD-163'],
 'CIZ2001-0121': ['PD-074', 'PD-153', 'PD-CC', 'PD-163'],
 'CIZ2001-0122': ['PD-074', 'PD-153', 'PD-CC', 'PD-163', 'PD-160'],
 'CIZ2002-0005': ['PD-074', 'PD-061', 'PD-CC', 'PD-160', 'PD

In [1520]:
assignment_map_data = create_assignment_map_data(input_data, output_data, result_mapping_data, lat_lng_dict)
# assignment_map_data["mapping_lines"]

In [1521]:
elements_settings = assignment_elements_settings()
# elements_settings

In [1522]:
display_settings = folium_map_display_settings(map_center)
# display_settings

In [1523]:
legend_html = create_legend_text_to_folium_map(input_data, output_data)
# legend_html

In [1524]:
folium_markers = create_markers_for_assignment_map(assignment_map_data, elements_settings["icon_settings"])
# folium_markers

In [1525]:
has_line = False
folium_lines = create_lines_for_assignment_map(assignment_map_data, elements_settings["line_settings"]) if has_line else []
# folium_lines

In [1526]:
# create_visualization_2(input_data, output_data, has_line)

In [1527]:
folium_map = get_visualization_2(display_settings, folium_markers, folium_lines, legend_html)
# folium_map

In [1528]:
depot_to_pd_mapping = get_depot_to_pd_mapping(input_data)
# depot_to_pd_mapping

In [1529]:
def create_full_customer_options(result_mapping_data):
    customer_options = []
    for customer in result_mapping_data['Customer_to_Phy_Depot']:
        customer_options.append({'label': customer, 'value': customer})
    return customer_options

# create_full_customer_options(result_mapping_data)

In [1530]:
def create_full_depot_options(result_mapping_data):
    depot_options = []
    for depot in result_mapping_data['TCO_Depot_to_Customer']:
        depot_options.append({'label': depot, 'value': depot})
    return depot_options

# create_full_depot_options(result_mapping_data)

In [1531]:
def create_full_factory_options(result_mapping_data):
    factory_options = []
    for factory in result_mapping_data['Factory_to_Phy_Depot']:
        factory_options.append({'label': factory, 'value': factory})
    return factory_options

# create_full_factory_options(result_mapping_data)

In [1532]:
def get_owner_and_customer_code(input_data):
    owner_mapping = {}

    for customer in input_data["customers"]:
        owner = customer["customerDetails"]["cType"]["typeOfCustomerByOwner"]
        customer_code = customer["customerCode"]

        if owner not in owner_mapping:
            owner_mapping[owner] = []

        owner_mapping[owner].append(customer_code)

    return owner_mapping



In [1533]:
owner_customer_mapping = get_owner_and_customer_code(input_data)
# owner_customer_mapping

In [1534]:
def create_owner_options(input_data):
    owner_customer_mapping = get_owner_and_customer_code(input_data)

    owner_options = list(owner_customer_mapping.keys())
    return [{'label': owner, 'value': owner} for owner in owner_options]

# Sử dụng hàm để tạo options cho owner
owner_options = create_owner_options(input_data)
print(owner_options)


[{'label': 'ST', 'value': 'ST'}, {'label': 'MĐ', 'value': 'MĐ'}, {'label': 'SH', 'value': 'SH'}]


In [1535]:
def create_customer_options(owner_customer_mapping, selected_owner):
    if selected_owner is None:
        return []
    customer_codes = owner_customer_mapping.get(selected_owner, [])
    options = [{'label': code, 'value': code} for code in customer_codes]
    return options

In [1536]:
customer_codes = owner_customer_mapping.get('SH', [])
customer_codes
options = [{'label': code, 'value': code} for code in customer_codes]
# options

In [1537]:
def create_depot_options(result_mapping_data, selected_customer):
    if selected_customer is None:
        return []
    depot_codes = result_mapping_data['Customer_to_Phy_Depot'].get(selected_customer, [])
    options = [{'label': code, 'value': code} for code in depot_codes]
    return options

In [1538]:
depot_codes = result_mapping_data['Customer_to_Phy_Depot'].get('CIZ2002-0008', [])
depot_codes
# options = [{'label': code, 'value': code} for code in depot_codes]
# options


['PD-153', 'PD-061', 'PD-CC', 'PD-160', 'PD-162']

In [1539]:
def create_factory_options(result_mapping_data, selected_depot):
    if selected_depot is None:
        return []
    factory_codes = result_mapping_data['Phy_Depot_to_Factory'].get(selected_depot, [])
    options = [{'label': code, 'value': code} for code in factory_codes]
    return options

In [1540]:
factory_codes = result_mapping_data['Phy_Depot_to_Factory'].get('PD-162', [])
options = [{'label': code, 'value': code} for code in factory_codes]
options


[{'label': '061', 'value': '061'},
 {'label': '162', 'value': '162'},
 {'label': 'CC', 'value': 'CC'},
 {'label': '166', 'value': '166'}]

In [1541]:
depot_customer_mapping = result_mapping_data['Phy_Depot_to_Customer']
def create_depot_options(depot_customer_mapping):
    if not depot_customer_mapping:
        return []

    depot_options = list(depot_customer_mapping.keys())

    depot_options = list(set(depot_options))  # Remove duplicates
    depot_options = [{'label': depot, 'value': depot} for depot in depot_options]

    return depot_options
depot_options = create_depot_options(depot_customer_mapping)


In [1542]:
factory_depot_mapping = result_mapping_data['Factory_to_Phy_Depot']
def create_factory_options(factory_depot_mapping):
    if not factory_depot_mapping:
        return []

    factory_options = list(factory_depot_mapping.keys())

    factory_options = list(set(factory_options))  # Remove duplicates
    factory_options = [{'label': factory, 'value': factory} for factory in factory_options]

    return factory_options
factory_options = create_factory_options(factory_depot_mapping)

### View Dash

In [1543]:
app = dash.Dash(__name__)

In [1544]:
owner_to_customer = html.Div(id='select-from-owner', children=[
        dbc.Row([
        dbc.Col(
            html.Div([
                dcc.Dropdown(
                    id='owner-dropdown',
                    options=owner_options,
                    value=None,
                    placeholder="Chọn chủ sở hữu"
                ),
            ]),
            width=3
        ),
        dbc.Col(
            dcc.Dropdown(
                id='customer-dropdown-1',
                options=[],
                clearable=True,
                multi=True,
                placeholder="Chọn khách hàng từ chủ sở hữu"
            ),
            width=3
        ),
        dbc.Col(
            dcc.Dropdown(
                id='depot-dropdown-1',
                options=[],
                clearable=True,
                multi=True,
                placeholder="Chọn kho từ khách hàng"
            ),
            width=3
        ),
        dbc.Col(
            dcc.Dropdown(
                id='factory-dropdown-1',
                options=[],
                clearable=True,
                multi=True,
                placeholder="Chọn nhà máy từ kho"
            ),
            width=3
        )
    ]),
])

In [1545]:
depot_to_customer_and_factory = html.Div(id = 'select-from-depot', children=[
	dbc.Row([
        dbc.Col(
            dcc.Dropdown(
                id='depot-dropdown-2',
                options=depot_options,
                value=None,
                multi=True,
                placeholder="Chọn kho",
                style={'width': '100%', 'font-size': '13px', 'background-color': 'primary'}
            ),
            width=3
        ),
        dbc.Col(
            dcc.Dropdown(
                id='customer-dropdown-2',
                options=[],
                value=None,
                multi=True,
                placeholder="Chọn khách hàng từ kho",
                style={'width': '100%', 'font-size': '13px','background-color': 'primary'}
            ),
            width=3
        ),
        dbc.Col(
            dcc.Dropdown(
                id='factory-dropdown-2',
                options=[],
                value=None,
                multi=True,
                placeholder="Chọn nhà máy từ kho",
                style={'width': '100%', 'font-size': '13px', 'background-color': 'primary'}
            )
        )
    ])
])

In [1546]:
factory_to_depot = html.Div(id='select-from-factory',children=[
	dbc.Row([
        dbc.Col(
            dcc.Dropdown(
                id='factory-dropdown-3',
                options=factory_options,
                value=None,
                multi=True,
                placeholder="Chọn nhà máy",
                style={'width': '100%', 'font-size': '13px', 'background-color': 'primary'}
            )
        ),
        dbc.Col(
            dcc.Dropdown(
                id='depot-dropdown-3',
                options=[],
                value=None,
                multi=True,
                placeholder="Chọn kho từ nhà máy",
                style={'width': '100%', 'font-size': '13px', 'background-color': 'primary'}
            ),
            width=3
        ),
        dbc.Col(
            dcc.Dropdown(
                id='customer-dropdown-3',
                options=[],
                value=None,
                multi=True,
                placeholder="Chọn khách hàng từ kho",
                style={'width': '100%', 'font-size': '13px','background-color': 'primary'}
            ),
            width=3
        ),
    ])
])

In [1547]:
# Style for vehicle information display within route containers
VEHICLE_CARD_STYLE = {
    'padding': '4px 8px',  # Smaller padding to make the card smaller
    'background': '#17a2b8',  # Bootstrap-like information color
    'color': 'white',
    'borderRadius': '1px',
    'fontWeight': 'bold',
    'textAlign': 'left',  # Align text to the left
    'boxShadow': '0px 2px 4px rgba(0, 0, 0, 0.1)',  # Soft shadow for depth
    'marginBottom': '1px',  # Reduced margin from the route_container
    'marginRight': '1px',
    'width': 'fit-content',  # Shrink the width to fit its content
    'alignSelf': 'flex-start',  # Align to the left of flex container
    'flexShrink': '0', # Prevent the vehicle info from shrinking
}

# Style for request cards (To drag-n-drop)
REQUEST_CARD_STYLE = {
    'display': 'inline-flex',  # Make request cards inline elements
    'padding': '4px 8px',  # Reduced padding
    'margin': '2px',
    'borderRadius': '1px',
    'border': '1px solid #ccc',
    'fontWeight': 'bold',
    'textAlign': 'center',
    'color': 'white',
    # 'background': f'linear-gradient(45deg, #6db1ff, #{np.random.randint(100, 999)})',
    # 'color': 'white',
    # 'boxShadow': '2px 2px 10px rgba(0,0,0,0.1)',
    # 'flex': '0 1 auto',  # Flex property for flexible sizing
    # 'cursor': 'grab',  # Indicate that the cards are draggable
}

ROUTE_CONTAINER_STYLE = {
    'display': 'flex',  # Display as flex to align children in a row
    'flexDirection': 'row',  # Lay out children in a horizontal row
    'overflowX': 'auto',  # Allow horizontal scrolling
    'padding': '6px',
    'border': '1px dashed #808080',
    'borderRadius': '2px',
    'marginTop': '1px',
    'marginBottom': '6px',
    'boxShadow': '0 4px 8px rgba(0,0,0,0.05)',
    'alignItems': 'left',  # Center items vertically
    # 'background': 'white',
    # 'justifyContent': 'flex-start',  # Align items to the start of the container
    # 'whiteSpace': 'nowrap',  # Prevent wrapping of items
}

DRAG_AND_DROP_ZONE_STYLE = {
    'padding': '20px',
    'border': '2px solid #28a745',  # Changed to a green shade for a more vibrant look
    'borderRadius': '12px',
    'marginTop': '20px',
    'background': '#f9f9f9',  # Light grey background for subtle contrast
    'boxShadow': '0 6px 12px rgba(0,0,0,0.1)',  # Deeper shadow for a 3D effect
    'display': 'flex',
    'flexDirection': 'column',  # Align route containers in a column
    'gap': '20px',  # Space between route containers
    'overflowY': 'auto',  # Enable vertical scrolling
    'maxHeight': '600px',  # Maximum height of the container
}

VIS_INIT_ROUTES_BUTTON_STYLE = {
    'backgroundColor': 'cyan',
    'color': 'black',
    'border': 'none',
    'padding': '10px',
    'margin': '10px',
    'borderRadius': '5px',
    'fontWeight': 'bold',
    'fontSize': '16px',
    'cursor': 'pointer'
}

VIS_UPDATED_ROUTES_BUTTON_STYLE = {
    'backgroundColor': 'magenta',
    'color': 'white',
    'border': 'none',
    'padding': '10px',
    'margin': '10px',
    'borderRadius': '5px',
    'fontWeight': 'bold',
    'fontSize': '16px',
    'cursor': 'pointer'
}

GET_UPDATED_ROUTES_BUTTON_STYLE = {
    'backgroundColor': 'red',
    'color': 'white',
    'border': 'none',
    'padding': '10px',
    'margin': '10px',
    'borderRadius': '5px',
    'fontWeight': 'bold',
    'fontSize': '16px',
    'cursor': 'pointer'
}

RENDER_VIS_BUTTON_STYLE = {
    'backgroundColor': 'orange',
    'color': 'white',
    'border': 'none',
    'padding': '10px',
    'margin': '10px',
    'borderRadius': '5px',
    'fontWeight': 'bold',
    'fontSize': '16px',
    'cursor': 'pointer'
}

In [1548]:
#common_display_const
BUTTON_WIDTH = 3
TAB_SIDEBAR_WIDTH = 1
TAB_MAIN_CONTENT_WIDTH = 11

In [1549]:
tab_switches = {
    "Input Visualization": ["Folium map"],
    "Output Visualization": []
}
tab_plugins = {
    "Input Visualization": ["Folium map"],
    "Output Visualization": []
}

In [1550]:
def create_header():
    return dbc.Row(
        children=[
            html.Div(
                [
                    html.H1("DASHBOARD")
                ],
                className='header',
            )
        ],
    )

def create_input_upload_button():
    return dcc.Upload(
        id='upload-input-data',
        children=dbc.Button([html.Div(style={'height':'25%', 'width':'25%'}), "Upload JSON Input Data"],
                            id="button-upload-input-data", color="dark"),
        multiple=False  # Allow one file at a time
    )

def create_output_upload_button():
    return dcc.Upload(
        id='upload-output-data',
        children=dbc.Button([html.Div(style={'height':'25%', 'width':'25%'}), "Upload JSON Output Data"],
                            id="button-upload-output-data", color="dark"),
        multiple=False  # Allow one file at a time
    )

def create_input_upload_message_box():
    return dbc.Row(
        dbc.Col(html.Div(id='input-upload-message-box'), width={'size': 6}),
        className="mb-4"
    )

def create_output_upload_message_box():
    return dbc.Row(
        dbc.Col(html.Div(id='output-upload-message-box'), width={'size': 6}),
        className="mb-4"
    )

def create_validate_io_message_box():
    return dbc.Row(
        dbc.Col(html.Div(id='validate-io-message-box'), width={'size': 6}),
        className="mb-4"
    )

def create_render_visualization_message_box():
    return dbc.Row(
        dbc.Col(html.Div(id='render-visualization-message-box'), width={'size': 6}),
        className="mb-4"
    )


def create_tab_content_input_visualization(switches, plugins):
    tab_row = [
        dbc.Col(switches, width=TAB_SIDEBAR_WIDTH),
        dbc.Col(plugins, width=TAB_MAIN_CONTENT_WIDTH)
    ]

    # return dbc.Row(tab_row)
    return dbc.Tab(label="Input Visualization", children=dbc.Row(tab_row))

def create_tab_content_output_visualization(switches, plugins):
    tab_row = [
        dbc.Col(switches, width=TAB_SIDEBAR_WIDTH),
        dbc.Col(plugins, width=TAB_MAIN_CONTENT_WIDTH)
    ]
    return dbc.Tab(label="Output Visualization", children=dbc.Row(tab_row))

def create_layout(tab_switches, tab_plugins):
    header = create_header()
    input_upload_message_box = create_input_upload_message_box()
    output_upload_message_box = create_output_upload_message_box()
    validate_io_message_box = create_validate_io_message_box()
    render_visualization_message_box = create_render_visualization_message_box()

    first_row = dbc.Row([
        dbc.Col(header, width=8),
        # dbc.Col(
        #     [dbc.Row(input_upload_message_box, class_name="mt-0 mb-0"),
        #         dbc.Row(output_upload_message_box, class_name="mt-0 mb-0"),
        #         dbc.Row(validate_io_message_box, class_name="mt-0 mb-0"),
        #         dbc.Row(render_visualization_message_box, class_name="mt-0 mb-0")],
        #     width=4
        # )
    ])

    # Define Theme Changer
    theme_changer = ThemeChangerAIO(aio_id='theme-changer', button_props={"size": "l", "color": "light",
                                                                            "style": {"background-color": "black"}})

    # Define JSON / zip Input & Output Files Upload Button
    input_upload_button = create_input_upload_button()
    output_upload_button = create_output_upload_button()

    # Define Input Visualization Tab
    input_vis_tab = create_tab_content_input_visualization(tab_switches["Input Visualization"],
                                                                tab_plugins["Input Visualization"])

    # Define Output Visualization Tab
    output_vis_tab = create_tab_content_output_visualization(tab_switches["Output Visualization"],
                                                                    tab_plugins["Output Visualization"])

    # Create a combined button row with reduced spacing
    combined_button_row = dbc.Row([
        # dbc.Col(input_upload_button, width=2),
        # dbc.Col(output_upload_button, width=2, className="ml-1"),  # Reduce spacing using ml-1 class
        dbc.Col(theme_changer, width=2, className="ml-auto"),  # Align to right using ml-auto class
        dbc.Col(html.Link(rel='stylesheet', href=dbc.themes.BOOTSTRAP, id='theme'), width=2)
    ])

    layout_structure = dbc.Container(fluid=True, children = [
        # Header
        first_row,

        # Theme Changer and Upload Buttons in a single row
        combined_button_row,

        # # Placeholder for render visualization button
        # dbc.Row([dbc.Col("", width=2),
        #             dbc.Col(dbc.Button("Render Visualization", id="render-vis-button",
        #                             className="button-render-vis",
        #                             style=RENDER_VIS_BUTTON_STYLE), width=2)]),

        # # Main Tabs
        # dbc.Row(dbc.Tabs(id="main-tabs", children=[
        #     dbc.Tab(id='input-vis-tab', label="Input Visualization", tab_id="tab_input", children=input_vis_tab, ),
        #     dbc.Tab(id='output-vis-tab', label="Output Visualization", tab_id="tab_output",children=output_vis_tab)
        # ])),

        # # Hidden Data Storages
        # dbc.Row(html.Div(id='last-clicked-vis-button'),style={'display': 'none'}),
        # dbc.Row(html.Div(id='input-data-storage', style={'display': 'none'})),
        # dbc.Row(html.Div(id='output-data-storage', style={'display': 'none'})),
        # dbc.Row(html.Div(id='dummy-div', style={'display': 'none'})),
        # dbc.Row(dcc.Store(id='json-output-store')),
        # dbc.Row(dcc.Store(id='drag-and-drop-data-store')),
    ])

    return layout_structure


In [1551]:
app.layout = html.Div([
    create_layout(tab_switches, tab_plugins),

    # dcc.Tabs(id='tabs',
    #         children=[
    #             dcc.Tab(label='Folium map', value='folium_map_tab'),
    #             dcc.Tab(label='Scatter map', value='scatter_map_tab'),
    #             # dcc.Tab(label='Barchart', value='barchart-tab')
    #         ], 
    #         value='folium-map-tabs'),
             

	html.Label('Select Display Group:'),
	dcc.Dropdown(
		id='display-selector',
		options=[
			{'label': 'from_owner', 'value': 'from_owner'},
			{'label': 'from_depot', 'value': 'from_depot'},
			{'label': 'from_factory', 'value': 'from_factory'},
		],
		value='From_owner'
	),
    owner_to_customer,
    depot_to_customer_and_factory,
    factory_to_depot,

    dcc.RadioItems(
        id='radio-items',
        options=[
            {'label': 'No Line', 'value': False},
            {'label': 'Has Lines', 'value': True}
        ],
        value=False
    ),
    
    html.Div(id='folium-map-container'),

    dcc.Graph(id='scatter-map', figure=create_fig()),
])



### Controller Dash

In [1552]:
def callback_functions_from_owner():
	@app.callback(
		[Output('customer-dropdown-1', 'options')],
		[Input('owner-dropdown', 'value')]
	)
	def update_customer_options(selected_owner):
		if selected_owner is None:
			return [[]]
		customer_options = create_customer_options(owner_customer_mapping, selected_owner)

		return [customer_options]
	# @app.callback(
	# 	Output('customer-dropdown-1', 'value'),
	# 	Input('customer-dropdown-1', 'options')
	# )
	# def set_customer_values(options):
	# 	return [option['value'] for option in options]

	@app.callback(
		[Output('depot-dropdown-1', 'options')],
		[Input('customer-dropdown-1', 'value')]
	)
	def update_depot_options(selected_customer):
		if selected_customer is None:
			return [[]]

		depot_options = []
		for customer_code in selected_customer:
			if customer_code in result_mapping_data['Customer_to_Phy_Depot']:
				depots = result_mapping_data['Customer_to_Phy_Depot'][customer_code]
				depot_options.extend(depots)

		depot_options = list(set(depot_options))  # Remove duplicates
		depot_options = [{'label': depot, 'value': depot} for depot in depot_options]

		return [depot_options]

	@app.callback(
		Output('depot-dropdown-1', 'value'),
		Input('depot-dropdown-1', 'options')
	)
	def set_depot_values(options):
		return [option['value'] for option in options]

	@app.callback(
		[Output('factory-dropdown-1', 'options')],
		[Input('depot-dropdown-1', 'value')]
	)
	def update_factory_options(selected_depot):
		if selected_depot is None:
			return [[]]

		factory_options = []
		for depot_code in selected_depot:
			if depot_code in result_mapping_data['Phy_Depot_to_Factory']:
				factories = result_mapping_data['Phy_Depot_to_Factory'][depot_code]
				factory_options.extend(factories)

		factory_options = list(set(factory_options))  # Remove duplicates
		factory_options = [{'label': factory, 'value': factory} for factory in factory_options]

		return [factory_options]

	@app.callback(
		Output('factory-dropdown-1', 'value'),
		Input('factory-dropdown-1', 'options')
	)
	def set_factory_values(options):
		return [option['value'] for option in options]

In [1553]:
def callback_functions_from_depot():
	@app.callback(
		[Output('customer-dropdown-2', 'options')],
		[Input('depot-dropdown-2', 'value')]
	)
	def update_customer_options(selected_depot):
		if selected_depot is None:
			return [[]]

		customer_options = []
		for depot_code in selected_depot:
			if depot_code in result_mapping_data['Phy_Depot_to_Customer']:
				customers = result_mapping_data['Phy_Depot_to_Customer'][depot_code]
				customer_options.extend(customers)

		customer_options = list(set(customer_options))  # Remove duplicates
		customer_options = [{'label': customer, 'value': customer} for customer in customer_options]

		return [customer_options]


	@app.callback(
		Output('customer-dropdown-2', 'value'),
		Input('customer-dropdown-2', 'options')
	)
	def set_customer_values(options):
		return [option['value'] for option in options]


	@app.callback(
		[Output('factory-dropdown-2', 'options')],
		[Input('depot-dropdown-2', 'value')]
	)
	def update_factory_options(selected_depot):
		if selected_depot is None:
			return [[]]

		factory_options = []
		for depot_code in selected_depot:
			if depot_code in result_mapping_data['Phy_Depot_to_Factory']:
				factories = result_mapping_data['Phy_Depot_to_Factory'][depot_code]
				factory_options.extend(factories)

		factory_options = list(set(factory_options))  # Remove duplicates
		factory_options = [{'label': factory, 'value': factory} for factory in factory_options]

		return [factory_options]

	@app.callback(
		Output('factory-dropdown-2', 'value'),
		Input('factory-dropdown-2', 'options')
	)
	def set_factory_values(options):
		return [option['value'] for option in options]

In [1554]:
def callback_functions_from_factory():
	@app.callback(
		[Output('depot-dropdown-3', 'options')],
		[Input('factory-dropdown-3', 'value')]
	)
	def update_depot_options(selected_factory):
		if selected_factory is None:
			return [[]]

		depot_options = []
		for factory_code in selected_factory:
			if factory_code in result_mapping_data['Factory_to_Phy_Depot']:
				factories = result_mapping_data['Factory_to_Phy_Depot'][factory_code]
				depot_options.extend(factories)

		depot_options = list(set(depot_options))  # Remove duplicates
		depot_options = [{'label': depot, 'value': depot} for depot in depot_options]

		return [depot_options]

	@app.callback(
		Output('depot-dropdown-3', 'value'),
		Input('depot-dropdown-3', 'options')
	)
	def set_depot_values(options):
		return [option['value'] for option in options]

	@app.callback(
		Output('customer-dropdown-3', 'value'),
		Input('customer-dropdown-3', 'options')
	)
	def set_customer_values(options):
		return [option['value'] for option in options]


	@app.callback(
		[Output('customer-dropdown-3', 'options')],
		[Input('depot-dropdown-3', 'value')]
	)
	def update_customer_options(selected_depot):
		if selected_depot is None:
			return [[]]

		customer_options = []
		for depot_code in selected_depot:
			if depot_code in result_mapping_data['Phy_Depot_to_Customer']:
				customers = result_mapping_data['Phy_Depot_to_Customer'][depot_code]
				customer_options.extend(customers)

		customer_options = list(set(customer_options))  # Remove duplicates
		customer_options = [{'label': customer, 'value': customer} for customer in customer_options]

		return [customer_options]


In [1555]:
# @app.callback(
#     Output()
# )

In [1556]:
@app.callback(
    Output('select-from-owner', 'style'),
    Output('select-from-depot', 'style'),
    Output('select-from-factory', 'style'),
    Input('display-selector', 'value')
)
def toggle_display(selected_display):
    select_from_owner = {'display': 'block'} if selected_display == 'from_owner' else {'display': 'none'}
    select_from_depot = {'display': 'block'} if selected_display == 'from_depot' else {'display': 'none'}
    select_from_factory = {'display': 'block'} if selected_display == 'from_factory' else {'display': 'none'}

    return select_from_owner, select_from_depot, select_from_factory

In [1557]:
callback_functions_from_owner()
callback_functions_from_depot()
callback_functions_from_factory()

In [1558]:
def filter_data_by_owner(data, selected_owner):
    owner_mapping = get_owner_and_customer_code(data)
    if selected_owner in owner_mapping:
        selected_customers = owner_mapping[selected_owner]
    data['customers'] = [customer for customer in data['customers'] if customer['customerCode'] in selected_customers]
    return data

def filter_data_by_customer(data, selected_customers):
    data['customers'] = [customer for customer in data['customers'] if customer['customerCode'] in selected_customers]
    return data

def filter_data_by_depot(data, selected_depots):
    data['physicDepots'] = [depot for depot in data['physicDepots'] if depot['depotPhysicCode'] in selected_depots]
    return data

def filter_data_by_factory(data, selected_depots):
    data['factories'] = [factory for factory in data['factories'] if factory['factoryCode'] in selected_depots]
    return data

In [1559]:
@app.callback(
    Output('folium-map-container', 'children'),
    [
        # Input('tabs', 'value'),
        Input('display-selector', 'value'),
        Input('owner-dropdown', 'value'),
        Input('customer-dropdown-1', 'value'),
        Input('depot-dropdown-1', 'value'),
        Input('factory-dropdown-1', 'value'),
        Input('depot-dropdown-2', 'value'),
        Input('customer-dropdown-2', 'value'),
        Input('factory-dropdown-2', 'value'),
        Input('factory-dropdown-3', 'value'),
        Input('depot-dropdown-3', 'value'),
        Input('customer-dropdown-3', 'value'),
        Input('radio-items', 'value')
    ]
)
def update_folium_map(
    # selected_tab,
    selected_display,
    selected_owner,
    selected_customer_1,
    selected_depot_1,
    selected_factory_1,
    selected_depot_2,
    selected_customer_2,
    selected_factory_2,
    selected_factory_3,
    selected_depot_3,
    selected_customer_3,
    selected_line
):
    # if selected_tab == 'folium_map_tab':
        filtered_data = input_data.copy()
        selected_locations = []

        if selected_display == 'from_owner':
            if selected_owner:
                filtered_data = filter_data_by_owner(filtered_data, selected_owner)

            if selected_customer_1:
                filtered_data = filter_data_by_customer(filtered_data, selected_customer_1)
                selected_locations.extend(selected_customer_1)

            if selected_depot_1:
                filtered_data = filter_data_by_depot(filtered_data, selected_depot_1)
                selected_locations.extend(selected_depot_1)

            if selected_factory_1:
                filtered_data = filter_data_by_factory(filtered_data, selected_factory_1)
                selected_locations.extend(selected_factory_1)

        elif selected_display == 'from_depot':
            if selected_depot_2:
                filtered_data = filter_data_by_depot(filtered_data, selected_depot_2)
                selected_locations.extend(selected_depot_2)

            if selected_customer_2:
                filtered_data = filter_data_by_customer(filtered_data, selected_customer_2)
                selected_locations.extend(selected_customer_2)

            if selected_factory_2:
                filtered_data = filter_data_by_factory(filtered_data, selected_factory_2)
                selected_locations.extend(selected_factory_2)
        else:
            if selected_factory_3:
                filtered_data = filter_data_by_factory(filtered_data, selected_factory_3)
                selected_locations.extend(selected_factory_3)

            if selected_depot_3:
                filtered_data = filter_data_by_depot(filtered_data, selected_depot_3)
                selected_locations.extend(selected_depot_3)

            if selected_customer_3:
                filtered_data = filter_data_by_customer(filtered_data, selected_customer_3)
                selected_locations.extend(selected_customer_3)

        if selected_line:
            has_line = True
        else:
            has_line = False

        folium_map = create_visualization_2(filtered_data, output_data, has_line, selected_locations)

        return html.Iframe(srcDoc=folium_map._repr_html_(), width="100%", height="600")


In [1560]:
# Create callback to update plot
@app.callback(
    Output('scatter-map', 'figure'),
    [
    # Input('tabs','value'),
    Input('scatter-map', 'clickData'),
    State('scatter-map', 'figure')]
)
def update_scatter_map(clickData, figure):
    # if selected_tab == 'scatter_map_tab':
        fig = go.Figure(data=figure['data'], layout=figure['layout'])

        if not clickData:
            # Return empty figure if no click data is available
            return fig


        # Get the clicked location
        clicked_location = (clickData['points'][0]['lat'], clickData['points'][0]['lon'])


        # Initialize lists to hold Scattermapbox traces
        traces = []

        # Initialize a list to hold lines for the big trace
        big_trace_lines = []

        lat_lng_dict = list_of_dicts_to_dict_of_dicts(input_data["locations"], "locationCode")
        result_mapping_data = summarize_factory_depot_customer_mapping_sm(tco_to_physical_mapping, output_data["depots"])
        assignment_map_data = create_assignment_map_data_sm(input_data, output_data, result_mapping_data, lat_lng_dict)
        

        # Find all lines related to the clicked location and add them to the big trace
        for line in assignment_map_data['mapping_lines']:
            if line['type'] == 'phy_depot_to_customer' or line['type'] == 'customer_to_phy_depot':
                line_color = 'gray'  # Màu xám cho đường nối depot-customer
            elif line['type'] == 'phy_depot_to_factory' or line['type'] == 'factory_to_phy_depot':
                line_color = 'brown'  # Màu nâu cho đường nối depot-factory

            if line['start_coordinate'] == clicked_location:
                big_trace_lines.append(
                    go.Scattermapbox(
                        lat=[line['start_coordinate'][0], line['end_coordinate'][0]],
                        lon=[line['start_coordinate'][1], line['end_coordinate'][1]],
                        mode='lines',
                        line=dict(width=2, color = line_color),
                        name=f'Lines from {clicked_location}'
                    )
                )


        # Cập nhật big_trace_lines vào fig
        fig.update_traces(
            overwrite=True,
            visible=True,
            showlegend = False
        )
        # fig.add_traces(traces + big_trace_lines)  # Thêm big_trace_lines vào biểu đồ
        fig.add_traces(traces + big_trace_lines)  # Thêm big_trace_lines vào biểu đồ


        return fig

### App

In [1561]:
if __name__ == '__main__':
    app.run_server(port = 3000)